In [ ]:
# THIS IS A SCRIPT TO GRAB COORDINATES OF LANDMARKS FOR HAND GESTURE RECOGNITION PROJECT WITH MEDIAPIPE

In [ ]:
# STEP 1: IMPORTS

In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

In [ ]:
# STEP 2: MEDIAPIPE MODELS FOR MAPPING

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [ ]:
# STEP 3: SET A LIST TO STORE LANDMARK COORDINATES FOR GESTURE. WHEN COLLECTING DATA REMEMBER TO RERUN BEFORE NEW GESTURE

In [78]:
gesture_data = []

In [66]:
# NAME OF THE GESTURE THAT WE WILL BE COLLECTING FOR

In [79]:
current_gesture = "V"

In [68]:
# STEP 4: START CAMERA AND COLLECT LANDMARK COORDINATES AND GESTURE NAME TO GESTURE_DATA

In [80]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands_processor:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame.")
            break

        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands_processor.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Extract landmarks
                landmarks = [{'x': landmark.x, 'y': landmark.y, 'z': landmark.z} for landmark in hand_landmarks.landmark]
                # Append gesture name and landmarks to the list
                gesture_data.append({"gesture": current_gesture, "landmarks": landmarks})

                # Visualization
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow('Gesture Data Collection', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@2147.665] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
I0000 00:00:1712670978.104562   96785 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


In [81]:
len(gesture_data)

604

In [56]:
# STEP 5: PREPARE THE DATA TO MAKE THE DF, SEPARATE COORDINATES TO GET 3 COORDINATES PER NODE (X, Y, AND Z SEPARATE FOR EACH 'DOT')
        # SET COLUMN NAMES

In [82]:
data_for_df = []
for gesture_instance in gesture_data:
    gesture = gesture_instance['gesture']
    # Flatten landmarks into a single list
    landmarks_flat = [coord for landmark in gesture_instance['landmarks'] for coord in (landmark['x'], landmark['y'], landmark['z'])]
    data_for_df.append([gesture] + landmarks_flat)

In [83]:
columns = ['gesture']
for i in range(21):
    columns += [f'landmark_{i}_x', f'landmark_{i}_y', f'landmark_{i}_z']

In [84]:
df_v = pd.DataFrame(data_for_df, columns=columns)

In [ ]:
# STEP 6: AFTER MAKING SEVERAL DFS WE MERGE THEM, SHUFFLE THE NEW DF AND SAVE IT TO A CSV FILE

In [85]:
alphabet_df = pd.read_csv('/Users/igazielinska/Downloads/PYTHON/MEDIAPIPE/asl_alphabet.csv')

In [86]:
alphabet_df = pd.concat([alphabet_df, df_v], ignore_index=True)

In [87]:
alphabet_df = alphabet_df.sample(frac = 1,ignore_index=True)

In [88]:
alphabet_df.to_csv('asl_alphabet.csv')

In [89]:
alphabet_df.head()

,Unnamed: 0.1,Unnamed: 0,gesture,landmark_0_x,landmark_0_y,landmark_0_z,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,...,landmark_17_z,landmark_18_x,landmark_18_y,landmark_18_z,landmark_19_x,landmark_19_y,landmark_19_z,landmark_20_x,landmark_20_y,landmark_20_z
0,23826.0,4271.0,H,0.866091,0.498706,-6.693230e-09,0.840791,0.408861,0.004693,0.794223,...,-0.060427,0.721388,0.478011,-0.067975,0.730407,0.486134,-0.058071,0.749075,0.488189,-0.049318
1,19344.0,12124.0,K,0.845493,0.734582,8.177098e-08,0.809632,0.662445,-0.010806,0.795381,...,-0.030272,0.877124,0.563922,-0.047902,0.855343,0.601193,-0.044417,0.838121,0.634099,-0.036603
2,19893.0,25162.0,I,0.798377,0.665993,-2.597428e-07,0.758528,0.630093,-0.017620,0.729099,...,-0.010558,0.801362,0.374532,-0.023177,0.794819,0.325176,-0.025836,0.787695,0.278156,-0.023571
3,23150.0,731.0,E,0.852711,0.691479,-3.858155e-07,0.790054,0.639891,-0.014217,0.750762,...,-0.037936,0.898595,0.321717,-0.052149,0.882797,0.330354,-0.053624,0.872871,0.374883,-0.051688
4,27890.0,10852.0,V,0.712514,0.975692,5.156759e-07,0.662651,0.932474,-0.011731,0.624600,...,-0.056432,0.710385,0.724654,-0.080260,0.679250,0.764386,-0.078445,0.661532,0.804750,-0.071294
